In [10]:
from pony.orm import *
import os
import json
import redis
import rq
import configparser
import utils

from utils import get_xxhash, get_exif
import os 
import get_files
import py.path

In [11]:
config = configparser.ConfigParser()
config.read('config.ini')

exif_cache = redis.StrictRedis(
    host=config["redis"]["host"],
    port=config["redis"]["port"],
    db=config["redis"]["exif"],
)
xxhash_cache = redis.StrictRedis(
    host=config["redis"]["host"],
    port=config["redis"]["port"],
    db=config["redis"]["xxhash"],
)

In [12]:
import_dir = "/vault/Pictures/2002"

IMAGE_EXT = [".cr2", ".dng", ".jpg"]
VIDEO_EXT = [".mp4"]

media_files = get_files.get_files(import_dir)

In [21]:
for media_file in media_files:
    xxhash=xxhash_cache.get(media_file)
    break

In [22]:
xxhash is None

True

In [23]:
xxhash = get_xxhash(media_file)
xxhash

'496c8cbe4c0c73ea'

In [24]:
xxhash_cache.set(media_file, xxhash)

True

In [25]:
xxhash_cache.get(media_file)

b'496c8cbe4c0c73ea'

In [39]:
def cache_xxhash(media_file):
    if xxhash_cache.get(str(media_file)) is None:
        xxhash = get_xxhash(str(media_file))
        xxhash_cache.set(str(media_file), xxhash)
        print("Caching hash: {}".format(media_file))
    else:
        print("Cached hash : {}".format(media_file))        
def cache_exif(media_file):
    if exif_cache.get(xxhash) is None:
        EXIF = et.get_metadata(str(media_file))
        exif_cache.set(xxhash, json.dumps(EXIF))
        print("Caching EXIF: {}".format(media_file))
    else:
        print("Cached EXIF : {}".format(media_file))

In [37]:
cache_xxhash(media_file)

Cached : /vault/Pictures/2002/Thanksgiving 02/021128-142824.jpg


In [30]:
for media_file in media_files:
    xxhash = utils.get_xxhash(str(media_file))
    if xxhash_cache.get(str(media_file)) is None:
        xxhash_cache.set(str(media_file), xxhash)
        print("Caching: {}".format(media_file))
    else:
        print("Cached : {}".format(media_file))
    break

Caching: /vault/Pictures/2002/Thanksgiving 02/021128-142824.jpg
